In [5]:
import requests
import folium
import json
import pystac
import rioxarray
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl
import fsspec
import geopandas as gpd

In [6]:
sas_token = "se=2023-02-13T19%3A15Z&sp=racwdl&sv=2018-11-09&sr=c&skoid=b4d39ccb-ca9c-4e9d-a183-5e0f3ba3e609&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-02-06T19%3A16%3A18Z&ske=2023-02-13T19%3A15%3A00Z&sks=b&skv=2018-11-09&sig=z/AIiFdLunJ/tg9i/gJUqvLRgRxghWgXiIyAm1sV2Zo%3D"

In [7]:
#URL = f'https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/10TES/runoff_onset_10TES_allyears_median_40m.tif?{sas_token}'
#URL = f'https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/10TES/runoff_onset_10TES_allyears_median_40m.tif'
#da = rioxarray.open_rasterio(URL)
#da
#f,ax=plt.subplots()
#da.plot(ax=ax)
#ctx.add_basemap(ax=ax, crs=da.rio.crs, source=ctx.providers.Stamen.Terrain)

In [8]:
url = 'https://github.com/scottyhq/mgrs/raw/main/MGRS_LAND.parquet' # Scott created an MGRS parquet file here https://github.com/scottyhq/mgrs
with fsspec.open(url) as file:
    mgrs_gdf = gpd.read_parquet(file)
tile_names_10T = list(mgrs_gdf[mgrs_gdf.tile.str.match('10T[DEFG]')].tile)
tile_names = open('../sar_snow_melt_timing/input/MGRS_square_list/square_list.txt','r').readlines()
tile_names = [i.rstrip('\n') for i in tile_names]
tile_names = [tile for tile in tile_names if tile in tile_names_10T]
tile_names

['10TGM',
 '10TGL',
 '10TGP',
 '10TGN',
 '10TGQ',
 '10TFP',
 '10TFT',
 '10TFN',
 '10TGT',
 '10TFS',
 '10TFM',
 '10TGK',
 '10TFL',
 '10TES',
 '10TEQ',
 '10TDT',
 '10TEP',
 '10TEN',
 '10TFK',
 '10TEL',
 '10TDL',
 '10TGR',
 '10TFQ',
 '10TET',
 '10TDM',
 '10TGS',
 '10TEM',
 '10TFR',
 '10TER']

In [9]:
import branca
from branca.colormap import LinearColormap

cm = branca.colormap.linear.viridis.scale(0,365)
#cm.colors.reverse()
#cm = branca.colormap.LinearColormap(colors=cm.colors, caption='sqrt(gamma0)')
cm

In [10]:
m = folium.Map(
    location=(40.76, -111.89),
    tiles="Stamen Terrain", #"cartodbpositron"
    zoom_start=5,
)

interannual_medians = folium.FeatureGroup(name='2015-2022 Median Snowmelt Runoff Onset Date')
interannual_std = folium.FeatureGroup(name='2015-2022 Standard Deviation of Snowmelt Runoff Onset Date')
interannual_polyfit = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date Trend')
interannual_corr = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date Trend Strength')


for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_median_40m.tif"
    tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_medians.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, colormap=mpl.cm.viridis.colors,opacity=1,name='2015-2022 Median Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_std_40m.tif"
    tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
    expression = "rescale=0,365&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_std.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, colormap=mpl.cm.viridis.colors,opacity=1,name='2015-2022 STD Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_polyfit_40m.tif"
    tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
    expression = "rescale=-1,1&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_polyfit.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, colormap=mpl.cm.viridis.colors,opacity=1,name='2015-2022 Snowmelt Runoff Onset Trend', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
for tile in tile_names:
    tif = f"https://github.com/egagli/sar_snowmelt_timing_map/raw/main/MGRS/{tile}/runoff_onset_{tile}_allyears_corr_strength_40m.tif"
    tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
    expression = "rescale=-1,1&colormap_name=viridis&nodata=nan"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_corr.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, colormap=mpl.cm.viridis.colors,opacity=1,name='2015-2022 Snowmelt Runoff Onset Trend Strength', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))

interannual_medians.add_to(m)
interannual_std.add_to(m)
interannual_polyfit.add_to(m)
interannual_corr.add_to(m)


folium.LayerControl().add_to(m)
cm.add_to(m)

m

In [11]:
m.save("index.html")